## Notebook Merge_Datasets
This notebook takes the data from pi0, eta and bck previously separated into individual train/test-sets and creates a train-dataset and a test-dataset 

In [1]:
import numpy as np

In [2]:
#maybe fix following issue in the future:
#due to the split in previous notebook the size and NCell is missmatched
# this leads to more memory being used than necessary
# if memory becomes a problem one could split the merging of test and train 

In [3]:
#path to files
pi0_train = 'Data-Split/pi0_train.npz'
eta_train = 'Data-Split/eta_train.npz'
bck_train = 'Data-Split/bck_train.npz'

pi0_test = 'Data-Split/pi0_test.npz'
eta_test = 'Data-Split/eta_test.npz'
bck_test = 'Data-Split/bck_test.npz'

In [4]:
#Create dataloader
data_pi0_train = np.load(pi0_train, allow_pickle=True)
data_eta_train = np.load(eta_train, allow_pickle=True)
data_bck_train = np.load(bck_train, allow_pickle=True)

data_pi0_test = np.load(pi0_test, allow_pickle=True)
data_eta_test = np.load(eta_test, allow_pickle=True)
data_bck_test = np.load(bck_test, allow_pickle=True)

print(list(data_bck_train.keys()))

['Size', 'ClusterN', 'Cluster', 'ClusterTiming', 'ClusterE', 'ClusterPt', 'ClusterModuleNumber', 'ClusterType', 'ClusterRow', 'ClusterCol', 'ClusterM02', 'ClusterM20', 'ClusterDistFromVert', 'PartE', 'PartPt', 'PartEta', 'PartPhi', 'PartIsPrimary', 'PartPID']


## Calculate size of dataset

In [5]:
pi0_size_train = data_pi0_train['Size']
eta_size_train = data_eta_train['Size']
bck_size_train = data_bck_train['Size']

size_train = pi0_size_train.item() + eta_size_train.item() + bck_size_train.item()

In [6]:
pi0_size_test = data_pi0_test['Size']
eta_size_test = data_eta_test['Size']
bck_size_test = data_bck_test['Size']

size_test = pi0_size_test.item() + eta_size_test.item() + bck_size_test.item()

## Initialise arrays 
These arrays are initialised to fit the whole dataset

In [7]:
ClusterN_train = np.zeros((size_train), dtype=np.ubyte)
ClusterN_test = np.zeros((size_test), dtype=np.ubyte)

ClusterN_train[0:pi0_size_train] = data_pi0_train["ClusterN"]
ClusterN_train[pi0_size_train:pi0_size_train+eta_size_train] = data_eta_train["ClusterN"]
ClusterN_train[-bck_size_train:] = data_bck_train["ClusterN"]

ClusterN_test[0:pi0_size_test] = data_pi0_test["ClusterN"]
ClusterN_test[pi0_size_test:pi0_size_test+eta_size_test] = data_eta_test["ClusterN"]
ClusterN_test[-bck_size_test:] = data_bck_test["ClusterN"]

ClusN_train_max = np.max(ClusterN_train)
ClusN_test_max = np.max(ClusterN_test)
ClusN_max = np.max([ClusN_train_max, ClusN_test_max])

In [8]:
ClusN_test_max

13

In [9]:
Cluster_train = np.zeros((size_train, ClusN_max), dtype=np.float32)
ClusterTiming_train = np.zeros((size_train, ClusN_max), dtype=np.float32)
ClusterType_train = np.zeros((size_train), dtype=np.ubyte)
ClusterE_train = np.zeros((size_train), dtype=np.float32)
ClusterPt_train = np.zeros((size_train), dtype=np.float32)
ClusterModuleNumber_train = np.zeros((size_train, ClusN_max), dtype=np.ubyte)
ClusterCol_train = np.zeros((size_train, ClusN_max), dtype=np.ubyte)
ClusterRow_train = np.zeros((size_train, ClusN_max), dtype=np.ubyte)
ClusterM20_train = np.zeros((size_train), dtype=np.float32)
ClusterM02_train = np.zeros((size_train), dtype= np.float32)
ClusterDistFromVert_train = np.zeros((size_train), dtype=np.float32)
PartE_train = np.zeros((size_train), dtype=np.float32)
PartPt_train = np.zeros((size_train), dtype=np.float32)
PartEta_train = np.zeros((size_train), dtype=np.float32)
PartPhi_train = np.zeros((size_train), dtype=np.float32)
PartIsPrimary_train = np.zeros((size_train), dtype=bool)
PartPID_train = np.zeros((size_train), dtype=np.short)

In [10]:
Cluster_test = np.zeros((size_test, ClusN_max), dtype=np.float32)
ClusterTiming_test = np.zeros((size_test, ClusN_max), dtype=np.float32)
ClusterType_test = np.zeros((size_test), dtype=np.ubyte)
ClusterE_test = np.zeros((size_test), dtype=np.float32)
ClusterPt_test = np.zeros((size_test), dtype=np.float32)
ClusterModuleNumber_test = np.zeros((size_test, ClusN_max), dtype=np.ubyte)
ClusterCol_test = np.zeros((size_test, ClusN_max), dtype=np.ubyte)
ClusterRow_test = np.zeros((size_test, ClusN_max), dtype=np.ubyte)
ClusterM20_test = np.zeros((size_test), dtype=np.float32)
ClusterM02_test = np.zeros((size_test), dtype= np.float32)
ClusterDistFromVert_test = np.zeros((size_test), np.float32)
PartE_test = np.zeros((size_test), dtype=np.float32)
PartPt_test = np.zeros((size_test), dtype=np.float32)
PartEta_test = np.zeros((size_test), dtype=np.float32)
PartPhi_test = np.zeros((size_test), dtype=np.float32)
PartIsPrimary_test = np.zeros((size_test), dtype=bool)
PartPID_test = np.zeros((size_test), dtype=np.short)

In [11]:
Cluster_test.shape

(47, 15)

# Load data into arrays

In [12]:
def load_into(arr, keyword, dataset):
    if dataset == 'train':
        if arr.ndim == 2:
            if arr.dtype == np.float32:
                arr[0:pi0_size_train] = np.lib.pad( data_pi0_train[keyword], ((0,0),(0, np.abs(data_pi0_train[keyword].shape[1] - ClusN_max))),'constant', constant_values=(np.NaN))
                arr[pi0_size_train:pi0_size_train+eta_size_train] = np.lib.pad( data_eta_train[keyword], ((0,0),(0, np.abs(data_eta_train[keyword].shape[1] - ClusN_max))),'constant', constant_values=(np.NaN))
                arr[-bck_size_train:] = np.lib.pad( data_bck_train[keyword], ((0,0),(0, np.abs(data_bck_train[keyword].shape[1] - ClusN_max))),'constant', constant_values=(np.NaN))
            else:
                arr[0:pi0_size_train] = np.lib.pad( data_pi0_train[keyword], ((0,0),(0, np.abs(data_pi0_train[keyword].shape[1] - ClusN_max))),'constant', constant_values=(100))
                arr[pi0_size_train:pi0_size_train+eta_size_train] = np.lib.pad( data_eta_train[keyword], ((0,0),(0, np.abs(data_eta_train[keyword].shape[1] - ClusN_max))),'constant', constant_values=(100))
                arr[-bck_size_train:] = np.lib.pad( data_bck_train[keyword], ((0,0),(0, np.abs(data_bck_train[keyword].shape[1] - ClusN_max))),'constant', constant_values=(100))

        else:        
            arr[0:pi0_size_train] = data_pi0_train[keyword]
            arr[pi0_size_train:pi0_size_train+eta_size_train] = data_eta_train[keyword]
            arr[-bck_size_train:] = data_bck_train[keyword]
    
    if dataset == 'test':
        if arr.ndim == 2:
            if arr.dtype == np.float32:
                arr[0:pi0_size_test] = np.lib.pad( data_pi0_test[keyword], ((0,0),(0, np.abs(data_pi0_test[keyword].shape[1] - ClusN_max))),'constant', constant_values=(np.NaN))
                arr[pi0_size_test:pi0_size_test+eta_size_test] = np.lib.pad( data_eta_test[keyword], ((0,0),(0, np.abs(data_eta_test[keyword].shape[1] - ClusN_max))),'constant', constant_values=(np.NaN))
                arr[-bck_size_test:] = np.lib.pad( data_bck_test[keyword], ((0,0),(0, np.abs(data_bck_test[keyword].shape[1] - ClusN_max))),'constant', constant_values=(np.NaN))
            else:
                arr[0:pi0_size_test] = np.lib.pad( data_pi0_test[keyword], ((0,0),(0, np.abs(data_pi0_test[keyword].shape[1] - ClusN_max))),'constant', constant_values=(100))
                arr[pi0_size_test:pi0_size_test+eta_size_test] = np.lib.pad( data_eta_test[keyword], ((0,0),(0, np.abs(data_eta_test[keyword].shape[1] - ClusN_max))),'constant', constant_values=(100))
                arr[-bck_size_test:] = np.lib.pad( data_bck_test[keyword], ((0,0),(0, np.abs(data_bck_test[keyword].shape[1] - ClusN_max))),'constant', constant_values=(100))    
        else:        
            arr[0:pi0_size_test] = data_pi0_test[keyword]
            arr[pi0_size_test:pi0_size_test+eta_size_test] = data_eta_test[keyword]
            arr[-bck_size_test:] = data_bck_test[keyword]

In [13]:
load_into(Cluster_train, 'Cluster', 'train')
load_into(ClusterTiming_train, 'ClusterTiming', 'train')
load_into(ClusterType_train, 'ClusterType', 'train')
load_into(ClusterE_train, 'ClusterE', 'train')
load_into(ClusterPt_train, 'ClusterPt', 'train')
load_into(ClusterModuleNumber_train, 'ClusterModuleNumber', 'train')
load_into(ClusterRow_train, 'ClusterRow', 'train')
load_into(ClusterCol_train, 'ClusterCol', 'train')
load_into(ClusterM20_train, 'ClusterM20','train')
load_into(ClusterM02_train, 'ClusterM02','train')
load_into(ClusterDistFromVert_train, 'ClusterDistFromVert', 'train')
load_into(PartE_train, 'PartE','train')
load_into(PartPt_train, 'PartPt','train')
load_into(PartEta_train, 'PartEta', 'train')
load_into(PartPhi_train, 'PartPhi', 'train')
load_into(PartIsPrimary_train, 'PartIsPrimary', 'train')
load_into(PartPID_train, 'PartPID', 'train')

In [14]:
load_into(Cluster_test, 'Cluster', 'test')
load_into(ClusterTiming_test, 'ClusterTiming', 'test')
load_into(ClusterType_test, 'ClusterType', 'test')
load_into(ClusterE_test, 'ClusterE', 'test')
load_into(ClusterPt_test, 'ClusterPt', 'test')
load_into(ClusterModuleNumber_test, 'ClusterModuleNumber', 'test')
load_into(ClusterCol_test, 'ClusterCol', 'test')
load_into(ClusterRow_test, 'ClusterRow', 'test')
load_into(ClusterM20_test, 'ClusterM20','test')
load_into(ClusterM02_test, 'ClusterM02','test')
load_into(ClusterDistFromVert_test, 'ClusterDistFromVert', 'test')
load_into(PartE_test, 'PartE','test')
load_into(PartPt_test, 'PartPt','test')
load_into(PartEta_test, 'PartEta', 'test')
load_into(PartPhi_test, 'PartPhi', 'test')
load_into(PartIsPrimary_test, 'PartIsPrimary', 'test')
load_into(PartPID_test, 'PartPID', 'test')

## Find Values for normalization
Minimum and maximum values from the train set, to normalize values in training to [0,1]

In [17]:
maxCellEnergy = np.nanmax(Cluster_train)
maxCellTiming = np.nanmax(ClusterTiming_train)
maxClusterEnergy = np.nanmax(ClusterE_train)
maxClusterPt = np.nanmax(ClusterPt_train)
maxClusterM20 = np.nanmax(ClusterM20_train)
maxClusterM02 = np.nanmax(ClusterM02_train)
maxClusterDistFromVert = np.nanmax(ClusterDistFromVert_train)
maxPartE = np.nanmax(PartE_train)
maxPartPt = np.nanmax(PartPt_train)
maxPartEta = np.nanmax(PartEta_train)
maxPartPhi = np.nanmax(PartPhi_train)

In [18]:
minCellEnergy = np.nanmin(Cluster_train)
minCellTiming = np.nanmin(ClusterTiming_train)
minClusterEnergy = np.nanmin(ClusterE_train)
minClusterPt = np.nanmin(ClusterPt_train)
minClusterM20 = np.nanmin(ClusterM20_train)
minClusterM02 = np.nanmin(ClusterM02_train)
minClusterDistFromVert = np.nanmin(ClusterDistFromVert_train)
minPartE = np.nanmin(PartE_train)
minPartPt = np.nanmin(PartPt_train)
minPartEta = np.nanmin(PartEta_train)
minPartPhi = np.nanmin(PartPhi_train)

## Save the train- and test-datasets and normalization for further usage

In [16]:
np.savez_compressed('../CNN/Data/data_train', Size = ClusterN_train.size, ClusterN=ClusterN_train
                    , Cluster=Cluster_train, ClusterTiming=ClusterTiming_train, ClusterE=ClusterE_train
                    , ClusterPt=ClusterPt_train, ClusterModuleNumber=ClusterModuleNumber_train
                    , ClusterType=ClusterType_train, ClusterCol=ClusterCol_train, ClusterRow=ClusterRow_train
                    , ClusterM02=ClusterM02_train, ClusterM20=ClusterM20_train
                    , ClusterDistFromVert=ClusterDistFromVert_train
                    , PartE=PartE_train, PartPt=PartPt_train, PartEta=PartEta_train, PartPhi=PartPhi_train
                    , PartIsPrimary=PartIsPrimary_train, PartPID=PartPID_train)

In [17]:
np.savez_compressed('../CNN/Data/data_test', Size = ClusterN_test.size, ClusterN=ClusterN_test, Cluster=Cluster_test
                    , ClusterTiming=ClusterTiming_test, ClusterE=ClusterE_test, ClusterPt=ClusterPt_test
                    , ClusterModuleNumber=ClusterModuleNumber_test, ClusterType=ClusterType_test
                    , ClusterCol=ClusterCol_test, ClusterRow=ClusterRow_test, ClusterM02=ClusterM02_test
                    , ClusterM20=ClusterM20_test, ClusterDistFromVert=ClusterDistFromVert_test
                    , PartE=PartE_test, PartPt=PartPt_test, PartEta=PartEta_test, PartPhi=PartPhi_test
                    , PartIsPrimary=PartIsPrimary_test, PartPID=PartPID_test)

In [22]:
np.savez_compressed('../CNN/Data/normalization', maxCellEnergy = maxCellEnergy, maxCellTiming = maxCellTiming
                    , maxClusterE = maxClusterEnergy, maxClusterPt = maxClusterPt, maxClusterM20 = maxClusterM20
                    , maxClusterM02 = maxClusterM02, maxClusterDistFromVert = maxClusterDistFromVert
                    , maxPartE = maxPartE, maxPartPt = maxPartPt, maxPartEta = maxPartEta, maxPartPhi= maxPartPhi
                    , minCellEnergy = minCellEnergy, minCellTiming = minCellTiming
                    , minClusterE = minClusterEnergy, minClusterPt = minClusterPt
                    , minClusterM20 = minClusterM20, minClusterM02 = minClusterM02
                    , minClusterDistFromVert = minClusterDistFromVert, minPartE = minPartE, minPartPt = minPartPt
                    , minPartEta = minPartEta, minPartPhi = minPartPhi)